In [1]:
import torch
import numpy as np
import pandas as pd
import mzcn as mz
print('matchzoo version', mz.__version__)

C:\Users\Administrator\Anaconda3\lib\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.11) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


matchzoo version 1.0.2


In [2]:
classification_task = mz.tasks.Classification(num_classes=2)
classification_task.metrics = ['acc']
print("`classification_task` initialized with metrics", classification_task.metrics)

`classification_task` initialized with metrics [accuracy]


In [3]:
def load_data(tmp_data,tmp_task):
	df_data = mz.pack(tmp_data,task=tmp_task)
	return df_data

In [5]:
print('data loading ...')
##数据集，并且进行相应的预处理
train=pd.read_csv('./data/train_data.csv').sample(100)
dev=pd.read_csv('./data/dev_data.csv').sample(50)
test=pd.read_csv('./data/test_data.csv').sample(30)
train_pack_raw = load_data(train,classification_task)
dev_pack_raw = load_data(dev,classification_task)
test_pack_raw=load_data(test,classification_task)
print('data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`')

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [6]:
import os
folder=mz.__path__[0]+'\\preprocessors\\units\\'
file=folder+'stopwords.txt'
if not os.path.exists(file):
    print('请将stopwords.txt放置在'+folder+'下面'+'否则会报错')
else:
    print('停用表配置成功')

停用表配置成功


In [7]:
import gc
gc.collect()

187

In [8]:
preprocessor = mz.models.ESIM.get_default_preprocessor()

In [9]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)

Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval:   0%| | 0/95 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.215 seconds.
Prefix dict has been built succesfully.
Processing text_left with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 95/95 [00:01<00:00, 53.52it/s]
Processing text_right with chain_transform of ChineseRemoveBlack => ChineseSimplified => ChineseEmotion => IsChinese => ChineseStopRemoval => ChineseTokenizeDemo => Tokenize => Lowercase => PuncRemoval: 100%|█| 98/98 [00:00<00:00, 187.84it/s]
Building Vocabulary from a datapack.: 100%|██████████████████████████████████████| 9

In [10]:
preprocessor.context

{'embedding_input_dim': 338,
 'filter_unit': <mzcn.preprocessors.units.frequency_filter.FrequencyFilter at 0x275b536a128>,
 'vocab_size': 338,
 'vocab_unit': <mzcn.preprocessors.units.vocabulary.Vocabulary at 0x275b58002b0>}

In [11]:
# glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=100)
# term_index = preprocessor.context['vocab_unit'].state['term_index']
# embedding_matrix = glove_embedding.build_matrix(term_index)
# l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
# embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [12]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='point'
)
devset = mz.dataloader.Dataset(
    data_pack=dev_pack_processed,
    mode='point'
)

In [13]:
padding_callback = mz.models.ESIM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
devloader = mz.dataloader.DataLoader(
    dataset=devset,
    stage='dev',
    callback=padding_callback
)

In [14]:
model = mz.models.ESIM()

model.params['task'] = classification_task
# model.params['embedding'] = embedding_matrix #这里是当加载glove等模型时取消该行注释
#设置embedding系数
model.params["embedding_output_dim"]=100
model.params["embedding_input_dim"]=preprocessor.context["embedding_input_dim"]
model.params['mask_value'] = 0
model.params['dropout'] = 0.2
model.params['hidden_size'] = 200
model.params['lstm_layer'] = 1

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

ESIM(
  (embedding): Embedding(338, 100, padding_idx=0)
  (rnn_dropout): RNNDropout(p=0.2, inplace=False)
  (input_encoding): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(100, 100, bidirectional=True)
    )
  )
  (attention): BidirectionalAttention()
  (projection): Sequential(
    (0): Linear(in_features=800, out_features=200, bias=True)
    (1): ReLU()
  )
  (composition): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(200, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=800, out_features=200, bias=True)
    (2): Tanh()
    (3): Dropout(p=0.2, inplace=False)
  )
  (out): Linear(in_features=200, out_features=2, bias=True)
)
Trainable params:  757802


In [15]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=devloader,
    validate_interval=None,
    epochs=5
)

In [16]:
trainer.run()

[Iter-3 Loss-0.696]:
  Validation: accuracy: 0.2



[Iter-6 Loss-0.695]:
  Validation: accuracy: 0.6286



[Iter-9 Loss-0.692]:
  Validation: accuracy: 0.8571



[Iter-12 Loss-0.688]:
  Validation: accuracy: 0.8571



[Iter-15 Loss-0.687]:
  Validation: accuracy: 0.8571

Cost time: 8.31123399734497s


In [17]:
import gc
gc.collect()

416